# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [15]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nhl_correlation(): 
    nhl_df=pd.read_csv("assets/nhl.csv")
    nhl_df['team'] = nhl_df['team'].str.replace('*', '')
    nhl_df = nhl_df[nhl_df['year']== 2018]
    nhl_df.drop(nhl_df[nhl_df['W'] == nhl_df['L']].index, inplace=True)
    nhl_df['team'] = nhl_df['team'].str.split(' ').apply(lambda x: x[-1])
    nhl_df = nhl_df[['team', 'W', 'L']]
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population', 'Metropolitan area': 'Metro'})
    cities = cities[['Metro', 'Population', 'NHL']]
    cities['NHL'] = cities['NHL'].str.replace(r'(\[.*\])|(—)', '').replace('', np.nan)
    cities = cities.dropna()
    cities['NHL'] = cities['NHL'].apply(lambda x: re.sub(r'(?<!^)(?=[A-Z])', '_', x))
    cities['NHL'] = cities['NHL'].str.replace(' _', ' ')
    # Preform operations to create team and merge with win/loss
    s = cities['NHL'].str.split('_').apply(pd.Series, 1).stack()
    s.index = s.index.droplevel(-1)
    s.name = 'team'
    teams = cities.join(s).astype({'Metro': str, 'Population': int, 'team': str})
    teams = teams[['Metro', 'Population', 'team']]
    teams['team'] = teams['team'].str.replace('[\w.]*\ ','')
    nhl_df = nhl_df.set_index('team')
    teams = teams.set_index('team')
    merged = nhl_df.join(teams, how='inner')
    merged = merged.astype({'W': int, 'L': int})
    merged['W/L'] = merged['W']/(merged['W'] + merged['L'])
    merged=merged.groupby('Metro').agg({'W/L': np.nanmean, 'Population': np.nanmean})
    population_by_region = merged['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = merged['W/L']  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [40]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nba_correlation(): 
    nba_df=pd.read_csv("assets/nba.csv")
    nba_df['team'] = nba_df['team'].str.replace(r'\(\d+\)', '')
    nba_df['team'] = nba_df['team'].str.replace('*', '')
    nba_df['team'] = nba_df['team'].str.strip()
    nba_df = nba_df[nba_df['year']== 2018]
    nba_df.drop(nba_df[nba_df['W'] == nba_df['L']].index, inplace=True)
    nba_df['team'] = nba_df['team'].str.split(' ').apply(lambda x: x[-1])
    nba_df = nba_df[['team', 'W', 'L']]
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population', 'Metropolitan area': 'Metro'})
    cities = cities[['Metro', 'Population', 'NBA']]
    cities['NBA'] = cities['NBA'].str.replace(r'(\[.*\])|(—)', '').replace('', np.nan)
    cities = cities.dropna()
    cities['NBA'] = cities['NBA'].apply(lambda x: re.sub(r'(?<!^)(?=[A-Z])', '_', x))
    cities['NBA'] = cities['NBA'].str.replace(' _', ' ')
    # Preform operations to create team and merge with win/loss
    s = cities['NBA'].str.split('_').apply(pd.Series, 1).stack()
    s.index = s.index.droplevel(-1)
    s.name = 'team'
    teams = cities.join(s).astype({'Metro': str, 'Population': int, 'team': str})
    teams = teams[['Metro', 'Population', 'team']]
    teams['team'] = teams['team'].str.replace('[\w.]*\ ','')
    nba_df = nba_df.set_index('team')
    teams = teams.set_index('team')
    merged = nba_df.join(teams, how='inner')
    merged = merged.astype({'W': int, 'L': int})
    merged['W/L'] = merged['W']/(merged['W'] + merged['L'])
    merged=merged.groupby('Metro').agg({'W/L': np.nanmean, 'Population': np.nanmean})
    population_by_region = merged['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = merged['W/L']  # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [88]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def mlb_correlation(): 
    mlb_df=pd.read_csv("assets/mlb.csv")
    mlb_df['team'] = mlb_df['team'].str.replace('*', '')
    mlb_df = mlb_df[mlb_df['year']== 2018]
    mlb_df['team'][0] = 'rsox'
    mlb_df['team'][8] = 'wsox'
    mlb_df.drop(mlb_df[mlb_df['W'] == mlb_df['L']].index, inplace=True)
    mlb_df['team'] = mlb_df['team'].str.split(' ').apply(lambda x: x[-1])
    mlb_df = mlb_df[['team', 'W', 'L']]
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population', 'Metropolitan area': 'Metro'})
    cities = cities[['Metro', 'Population', 'MLB']]
    cities['MLB'] = cities['MLB'].str.replace(r'(\[.*\])|(—)', '').replace('', np.nan)
    cities = cities.dropna()
    cities['MLB'] = cities['MLB'].apply(lambda x: re.sub(r'(?<!^)(?=[A-Z])', '_', x))
    cities['MLB'] = cities['MLB'].str.replace(' _', ' ')
    # Preform operations to create team and merge with win/loss
    s = cities['MLB'].str.split('_').apply(pd.Series, 1).stack()
    s.index = s.index.droplevel(-1)
    s.name = 'team'
    teams = cities.join(s).astype({'Metro': str, 'Population': int, 'team': str})
    teams = teams[['Metro', 'Population', 'team']]
    teams['team'] = teams['team'].str.replace('[\w.]*\ ','')
    mlb_df = mlb_df.set_index('team')
    teams = teams.set_index('team')
    teams = teams.reset_index()
    teams['team'][7] = 'wsox'
    teams['team'][11] = 'rsox'
    teams = teams.set_index('team')
    merged = mlb_df.join(teams, how='inner')
    merged = merged.astype({'W': int, 'L': int})
    merged['W/L'] = merged['W']/(merged['W'] + merged['L'])
    merged=merged.groupby('Metro').agg({'W/L': np.nanmean, 'Population': np.nanmean})
    population_by_region = merged['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = merged['W/L']  # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

0.15027698302669307


## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [89]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nfl_correlation(): 
    nfl_df=pd.read_csv("assets/nfl.csv")
    nfl_df['team'] = nfl_df['team'].str.replace('*', '')
    nfl_df['team'] = nfl_df['team'].str.replace('+', '')
    nfl_df = nfl_df[nfl_df['year']== 2018]
    nfl_df.drop(nfl_df[nfl_df['W'] == nfl_df['L']].index, inplace=True)
    nfl_df['team'] = nfl_df['team'].str.split(' ').apply(lambda x: x[-1])
    nfl_df = nfl_df[['team', 'W', 'L']]
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population', 'Metropolitan area': 'Metro'})
    cities = cities[['Metro', 'Population', 'NFL']]
    cities['NFL'] = cities['NFL'].str.replace(r'(\[.*\])|(—)', '').replace('', np.nan)
    cities = cities.dropna()
    cities['NFL'] = cities['NFL'].apply(lambda x: re.sub(r'(?<!^)(?=[A-Z])', '_', x))
    cities['NFL'] = cities['NFL'].str.replace(' _', ' ')
    # Preform operations to create team and merge with win/loss
    s = cities['NFL'].str.split('_').apply(pd.Series, 1).stack()
    s.index = s.index.droplevel(-1)
    s.name = 'team'
    teams = cities.join(s).astype({'Metro': str, 'Population': int, 'team': str})
    teams = teams[['Metro', 'Population', 'team']]
    teams['team'] = teams['team'].str.replace('[\w.]*\ ','')
    nfl_df = nfl_df.set_index('team')
    teams = teams.set_index('team')
    merged = nfl_df.join(teams, how='inner')
    merged = merged.astype({'W': int, 'L': int})
    merged['W/L'] = merged['W']/(merged['W'] + merged['L'])
    merged=merged.groupby('Metro').agg({'W/L': np.nanmean, 'Population': np.nanmean})
    population_by_region = merged['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = merged['W/L']  # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [96]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def sports_team_performance():
    def nhl_correlation(): 
        nhl_df=pd.read_csv("assets/nhl.csv")
        nhl_df['team'] = nhl_df['team'].str.replace('*', '')
        nhl_df = nhl_df[nhl_df['year']== 2018]
        nhl_df.drop(nhl_df[nhl_df['W'] == nhl_df['L']].index, inplace=True)
        nhl_df['team'] = nhl_df['team'].str.split(' ').apply(lambda x: x[-1])
        nhl_df = nhl_df[['team', 'W', 'L']]
        cities=pd.read_html("assets/wikipedia_data.html")[1]
        cities=cities.iloc[:-1,[0,3,5,6,7,8]]
        cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population', 'Metropolitan area': 'Metro'})
        cities = cities[['Metro', 'Population', 'NHL']]
        cities['NHL'] = cities['NHL'].str.replace(r'(\[.*\])|(—)', '').replace('', np.nan)
        cities = cities.dropna()
        cities['NHL'] = cities['NHL'].apply(lambda x: re.sub(r'(?<!^)(?=[A-Z])', '_', x))
        cities['NHL'] = cities['NHL'].str.replace(' _', ' ')
        # Preform operations to create team and merge with win/loss
        s = cities['NHL'].str.split('_').apply(pd.Series, 1).stack()
        s.index = s.index.droplevel(-1)
        s.name = 'team'
        teams = cities.join(s).astype({'Metro': str, 'Population': int, 'team': str})
        teams = teams[['Metro', 'Population', 'team']]
        teams['team'] = teams['team'].str.replace('[\w.]*\ ','')
        nhl_df = nhl_df.set_index('team')
        teams = teams.set_index('team')
        merged = nhl_df.join(teams, how='inner')
        merged = merged.astype({'W': int, 'L': int})
        merged['W/L'] = merged['W']/(merged['W'] + merged['L'])
        return merged.groupby('Metro').mean()
    def nba_correlation(): 
        nba_df=pd.read_csv("assets/nba.csv")
        nba_df['team'] = nba_df['team'].str.replace(r'\(\d+\)', '')
        nba_df['team'] = nba_df['team'].str.replace('*', '')
        nba_df['team'] = nba_df['team'].str.strip()
        nba_df = nba_df[nba_df['year']== 2018]
        nba_df.drop(nba_df[nba_df['W'] == nba_df['L']].index, inplace=True)
        nba_df['team'] = nba_df['team'].str.split(' ').apply(lambda x: x[-1])
        nba_df = nba_df[['team', 'W', 'L']]
        cities=pd.read_html("assets/wikipedia_data.html")[1]
        cities=cities.iloc[:-1,[0,3,5,6,7,8]]
        cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population', 'Metropolitan area': 'Metro'})
        cities = cities[['Metro', 'Population', 'NBA']]
        cities['NBA'] = cities['NBA'].str.replace(r'(\[.*\])|(—)', '').replace('', np.nan)
        cities = cities.dropna()
        cities['NBA'] = cities['NBA'].apply(lambda x: re.sub(r'(?<!^)(?=[A-Z])', '_', x))
        cities['NBA'] = cities['NBA'].str.replace(' _', ' ')
        # Preform operations to create team and merge with win/loss
        s = cities['NBA'].str.split('_').apply(pd.Series, 1).stack()
        s.index = s.index.droplevel(-1)
        s.name = 'team'
        teams = cities.join(s).astype({'Metro': str, 'Population': int, 'team': str})
        teams = teams[['Metro', 'Population', 'team']]
        teams['team'] = teams['team'].str.replace('[\w.]*\ ','')
        nba_df = nba_df.set_index('team')
        teams = teams.set_index('team')
        merged = nba_df.join(teams, how='inner')
        merged = merged.astype({'W': int, 'L': int})
        merged['W/L'] = merged['W']/(merged['W'] + merged['L'])
        return merged.groupby('Metro').mean()
    def mlb_correlation(): 
        mlb_df=pd.read_csv("assets/mlb.csv")
        mlb_df['team'] = mlb_df['team'].str.replace('*', '')
        mlb_df = mlb_df[mlb_df['year']== 2018]
        mlb_df['team'][0] = 'rsox'
        mlb_df['team'][8] = 'wsox'
        mlb_df.drop(mlb_df[mlb_df['W'] == mlb_df['L']].index, inplace=True)
        mlb_df['team'] = mlb_df['team'].str.split(' ').apply(lambda x: x[-1])
        mlb_df = mlb_df[['team', 'W', 'L']]
        cities=pd.read_html("assets/wikipedia_data.html")[1]
        cities=cities.iloc[:-1,[0,3,5,6,7,8]]
        cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population', 'Metropolitan area': 'Metro'})
        cities = cities[['Metro', 'Population', 'MLB']]
        cities['MLB'] = cities['MLB'].str.replace(r'(\[.*\])|(—)', '').replace('', np.nan)
        cities = cities.dropna()
        cities['MLB'] = cities['MLB'].apply(lambda x: re.sub(r'(?<!^)(?=[A-Z])', '_', x))
        cities['MLB'] = cities['MLB'].str.replace(' _', ' ')
        # Preform operations to create team and merge with win/loss
        s = cities['MLB'].str.split('_').apply(pd.Series, 1).stack()
        s.index = s.index.droplevel(-1)
        s.name = 'team'
        teams = cities.join(s).astype({'Metro': str, 'Population': int, 'team': str})
        teams = teams[['Metro', 'Population', 'team']]
        teams['team'] = teams['team'].str.replace('[\w.]*\ ','')
        mlb_df = mlb_df.set_index('team')
        teams = teams.set_index('team')
        teams = teams.reset_index()
        teams['team'][7] = 'wsox'
        teams['team'][11] = 'rsox'
        teams = teams.set_index('team')
        merged = mlb_df.join(teams, how='inner')
        merged = merged.astype({'W': int, 'L': int})
        merged['W/L'] = merged['W']/(merged['W'] + merged['L'])
        return merged.groupby('Metro').mean()
    def nfl_correlation(): 
        nfl_df=pd.read_csv("assets/nfl.csv")
        nfl_df['team'] = nfl_df['team'].str.replace('*', '')
        nfl_df['team'] = nfl_df['team'].str.replace('+', '')
        nfl_df = nfl_df[nfl_df['year']== 2018]
        nfl_df.drop(nfl_df[nfl_df['W'] == nfl_df['L']].index, inplace=True)
        nfl_df['team'] = nfl_df['team'].str.split(' ').apply(lambda x: x[-1])
        nfl_df = nfl_df[['team', 'W', 'L']]
        cities=pd.read_html("assets/wikipedia_data.html")[1]
        cities=cities.iloc[:-1,[0,3,5,6,7,8]]
        cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population', 'Metropolitan area': 'Metro'})
        cities = cities[['Metro', 'Population', 'NFL']]
        cities['NFL'] = cities['NFL'].str.replace(r'(\[.*\])|(—)', '').replace('', np.nan)
        cities = cities.dropna()
        cities['NFL'] = cities['NFL'].apply(lambda x: re.sub(r'(?<!^)(?=[A-Z])', '_', x))
        cities['NFL'] = cities['NFL'].str.replace(' _', ' ')
        # Preform operations to create team and merge with win/loss
        s = cities['NFL'].str.split('_').apply(pd.Series, 1).stack()
        s.index = s.index.droplevel(-1)
        s.name = 'team'
        teams = cities.join(s).astype({'Metro': str, 'Population': int, 'team': str})
        teams = teams[['Metro', 'Population', 'team']]
        teams['team'] = teams['team'].str.replace('[\w.]*\ ','')
        nfl_df = nfl_df.set_index('team')
        teams = teams.set_index('team')
        merged = nfl_df.join(teams, how='inner')
        merged = merged.astype({'W': int, 'L': int})
        merged['W/L'] = merged['W']/(merged['W'] + merged['L'])
        return merged.groupby('Metro').mean()

    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    nhl_nba = pd.merge(left = nhl_correlation(), right = nba_correlation(), how = 'inner', on = 'Metro')
    nhl_mlb = pd.merge(left = nhl_correlation(), right = mlb_correlation(), how = 'inner', on = 'Metro')
    nhl_nfl = pd.merge(left = nhl_correlation(), right = nfl_correlation(), how = 'inner', on = 'Metro')
    nba_mlb = pd.merge(left = nba_correlation(), right = mlb_correlation(), how = 'inner', on = 'Metro')
    nba_nfl = pd.merge(left = nba_correlation(), right = nfl_correlation(), how = 'inner', on = 'Metro')
    mlb_nfl = pd.merge(left = mlb_correlation(), right = nfl_correlation(), how = 'inner', on = 'Metro')
    
    p_nhl_nba = stats.ttest_rel(nhl_nba['W/L_x'], nhl_nba['W/L_y'])[1]
    p_nhl_mlb = stats.ttest_rel(nhl_mlb['W/L_x'], nhl_mlb['W/L_y'])[1]
    p_nhl_nfl = stats.ttest_rel(nhl_nfl['W/L_x'], nhl_nfl['W/L_y'])[1]
    p_nba_mlb = stats.ttest_rel(nba_mlb['W/L_x'], nba_mlb['W/L_y'])[1]
    p_nba_nfl = stats.ttest_rel(nba_nfl['W/L_x'], nba_nfl['W/L_y'])[1]
    p_mlb_nfl = stats.ttest_rel(mlb_nfl['W/L_x'], mlb_nfl['W/L_y'])[1]

    p_values = pd.DataFrame([[np.nan, p_nba_nfl, p_nhl_nfl, p_mlb_nfl], 
                             [p_nba_nfl, np.nan, p_nhl_nba, p_nba_mlb], 
                             [p_nhl_nfl, p_nhl_nba, np.nan, p_nhl_mlb], 
                             [p_mlb_nfl, p_nba_mlb, p_nhl_mlb, np.nan]], 
                            index = sports, columns = sports)                                                                
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values